In [ ]:
! nvidia-smi

In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
print("Đã cài đặt biến môi trường PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True")

In [ ]:
import torch
import os
from datasets import load_dataset, concatenate_datasets
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

DATA_PATH = "/root/indo_finetune_translation/data/"

# Đường dẫn đến hai tệp ngôn ngữ
FILE_EN = os.path.join(DATA_PATH, "WikiMatrix.en-id.en")
FILE_ID = os.path.join(DATA_PATH, "WikiMatrix.en-id.id")

# Tên model trên Hugging Face
MODEL_NAME = "SeaLLMs/SeaLLMs-v3-1.5B-Chat"

# Thư mục lưu kết quả (adapter)
# /kaggle/working/ là thư mục output của Kaggle
OUTPUT_DIR = "/root/indo_finetune_translation/data/results_finetune"

# Số lượng mẫu huấn luyện (để tránh quá 12 giờ)
# 500,000 là con số an toàn
NUM_SAMPLES = 150000

print(f"Đã cấu hình xong. Sẽ tải model: {MODEL_NAME}")
print(f"Sẽ tải dữ liệu từ: {DATA_PATH}")

In [ ]:
print("Bắt đầu tải và xử lý dữ liệu...")

# 1. Tải hai tệp riêng biệt
ds_en = load_dataset("text", data_files={"train": FILE_EN})['train']
ds_id = load_dataset("text", data_files={"train": FILE_ID})['train']

# Đổi tên cột
ds_en = ds_en.rename_column("text", "en_text")
ds_id = ds_id.rename_column("text", "id_text")

# 2. Ghép 2 bộ dữ liệu (yêu cầu số dòng bằng nhau)
if len(ds_en) != len(ds_id):
    raise ValueError(f"Hai tệp không có cùng số dòng! Tiếng Anh: {len(ds_en)}, Tiếng Indo: {len(ds_id)}")

dataset = concatenate_datasets([ds_en, ds_id], axis=1)
print(f"Đã ghép thành công! Tổng số cặp câu: {len(dataset)}")

# 3. Lấy mẫu (sample)
print(f"Đang lấy mẫu ngẫu nhiên {NUM_SAMPLES} cặp câu...")
dataset_sampled = dataset.shuffle(seed=42).select(range(NUM_SAMPLES))

# 4. Hàm format dữ liệu sang dạng Chat
def format_data_for_chat(example):
    eng_text = example['en_text']
    ind_text = example['id_text']

    # Chỉ xử lý nếu cả hai câu đều có nội dung
    if eng_text and ind_text:
        return {
            "messages": [
                {
                    "role": "user",
                    "content": f"Translate the following sentence from English to Indonesian: '{eng_text}'"
                },
                {
                    "role": "assistant",
                    "content": ind_text
                }
            ]
        }
    else:
        # Bỏ qua các cặp câu rỗng
        return None

# 5. Áp dụng hàm format
print("Đang định dạng dữ liệu sang dạng Chat...")
dataset_formatted = dataset_sampled.map(
    format_data_for_chat,
    remove_columns=['en_text', 'id_text']
)

# 6. Tách train/validation (95% train, 5% test)
dataset_split = dataset_formatted.train_test_split(test_size=0.05, seed=42)
train_dataset = dataset_split['train']
eval_dataset = dataset_split['test']

print("\n=== XỬ LÝ DỮ LIỆU HOÀN TẤT ===")
print(f"Số lượng mẫu huấn luyện: {len(train_dataset)}")
print(f"Số lượng mẫu đánh giá: {len(eval_dataset)}")
print("\n--- Mẫu dữ liệu đã định dạng ---")
print(train_dataset[0]['messages'])

In [ ]:
print(f"Đang tải model {MODEL_NAME} (chế độ 4-bit)...")

# 1. Cấu hình Quantization (QLoRA) - Chuyển sang 8-bit cho compute_dtype
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16, # Thay đổi sang float16
    bnb_4bit_use_double_quant=True,
)

# 2. Tải Model (đã được lượng tử hóa 4-bit)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    torch_dtype=torch.float16, # Explicitly set torch_dtype to float16
    device_map="auto", # Tự động phân bổ lên GPU
    trust_remote_code=True
)
model.config.use_cache = False # Tắt cache khi huấn luyện
print("Model đã được tải và lượng tử hóa 4-bit.")

# 3. Tải Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

# Sửa lỗi pad_token (model này không có sẵn)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

tokenizer.padding_side = "right" # Quan trọng cho model CausalLM

print("Tokenizer đã được tải và cấu hình.")
print("=== MODEL VÀ TOKENIZER SẴN SÀNG =====")

In [ ]:
print("Đang cấu hình LoRA (PEFT) và Training Arguments...")

# 1. Cấu hình LoRA (PEFT)
# Chúng ta chỉ định các lớp (layer) muốn "điều hợp"
peft_config = LoraConfig(
    lora_alpha=32,       # Độ "mạnh" của adapter
    lora_dropout=0.05,   # Tỷ lệ dropout
    r=16,                # Rank (càng cao càng phức tạp, 16 là mức tốt)
    bias="none",
    task_type="CAUSAL_LM",
    # (Quan trọng) Các module cần huấn luyện cho kiến trúc Qwen2
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj"
    ],
)

# 2. Cấu hình các tham số huấn luyện (Training Arguments)
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,                  # Thư mục lưu kết quả
    num_train_epochs=2,                     # 1 epoch là đủ cho SFT với dataset lớn
    per_device_train_batch_size=4,          # Giảm batch size cho GPU T4 (giảm còn 2 nếu OOM)
    gradient_accumulation_steps=8,          # Tăng tích lũy gradient để bù lại batch size nhỏ (2 * 8 = 16)
    eval_steps=500,                         # Đánh giá sau mỗi 500 bước
    save_steps=500,                         # Lưu checkpoint sau mỗi 500 bước
    logging_steps=50,                       # Log thông tin sau mỗi 50 bước
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    bf16=False,                              # Dùng bf16 (nhanh hơn trên GPU T4)
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    report_to="none",                     # Tắt báo cáo lên WandB/MLflow
)

# 3. Khởi tạo SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=peft_config,
    args=training_args
)

print("=== TRAINER ĐÃ SẴN SÀNG ===")

In [ ]:
print("Bắt đầu huấn luyện model...")
trainer.train()
print("Huấn luyện hoàn tất.")

In [ ]:
# Lưu model cuối cùng (chỉ là các adapter LoRA)
final_checkpoint_dir = os.path.join(OUTPUT_DIR, "final_checkpoint")
trainer.save_model(final_checkpoint_dir)

print(f"=== ĐÃ LƯU ADAPTER LoRA VÀO: {final_checkpoint_dir} ===")
# Bạn có thể tìm thấy thư mục này trong mục "Output" của Kaggle Version

In [ ]:
print("Đang tải model đã fine-tune để kiểm tra...")

# 1. Dọn dẹp bộ nhớ (nếu cần)
# del model
# del trainer
# torch.cuda.empty_cache()

# 2. Tải model gốc (Base model) 4-bit
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)

# 3. Tải adapter LoRA đã huấn luyện và ghép vào model gốc
# (Sử dụng đường dẫn đã lưu ở Cell 7)
model_finetuned = PeftModel.from_pretrained(base_model, final_checkpoint_dir)
print("Đã ghép adapter LoRA vào model gốc.")

# 4. (Rất nên làm) Hợp nhất (merge) để có tốc độ inference nhanh nhất
model_finetuned = model_finetuned.merge_and_unload()
print("Đã merge và unload adapter.")

# 5. Tải tokenizer (tải lại cho chắc)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# 6. Kiểm tra thử
print("\n=== KIỂM TRA DỊCH THUẬT ===")
test_sentence = "This new small language model is very powerful and efficient."
prompt_template = [
    {
        "role": "user",
        "content": f"Translate the following sentence from English to Indonesian: '{test_sentence}'"
    }
]

# Áp dụng chat template của model
prompt_text = tokenizer.apply_chat_template(prompt_template, tokenize=False, add_generation_prompt=True)

print(f"--- Prompt đầu vào:\n{prompt_text}")

# Tokenize
inputs = tokenizer(prompt_text, return_tensors="pt").to("cuda")

# Tạo văn bản
outputs = model_finetuned.generate(
    **inputs,
    max_new_tokens=100,
    pad_token_id=tokenizer.eos_token_id,
    temperature=0.7 # Thêm chút sáng tạo
)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Chỉ lấy phần trả lời của assistant
# (Model Qwen/SeaLLM dùng <|im_start|> và <|im_end|>)
assistant_response = response.split("<|im_start|>assistant\n")[-1]
# Bỏ <|im_end|> nếu có
assistant_response = assistant_response.replace("<|im_end|>", "").strip()

print(f"\n--- Câu gốc (EN):\n{test_sentence}")
print(f"\n--- Câu dịch (ID):\n{assistant_response}")